In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd
from datamatch import (
    ColumnsIndex, StringSimilarity, JaroWinklerSimilarity, DateSimilarity, ThresholdMatcher
)

from lib.columns import rearrange_complaint_columns
from lib.explore import print_df
from clean.baton_rouge_pd_cprr import clean_18
from clean.baton_rouge_csd_pprr import clean_17 as clean_csd_17, clean_19 as clean_csd_19
from clean.baton_rouge_pd_pprr import clean as clean_pprr

In [2]:
df17, df19 = clean_csd_17(), clean_csd_19()
df = clean_pprr()
df

,rank_desc,badge_no,last_name,first_name,middle_initial,data_production_year,agency,uid
0,cpl,291,abadie,jonathan,m,2021,Baton Rouge PD,572b5ea334efc3ff94da634ae323b930
1,cpl,457,acree,jason,k,2021,Baton Rouge PD,a02b3a98a05c9b8fedf038032d0b094e
2,off,1211,adams,aimee,,2021,Baton Rouge PD,b6f1552b3c0ef8c8b17562704971dbf0
3,off,1201,adams,dominique,,2021,Baton Rouge PD,48a35ff775efcb6cb53dbea5127a38fc
4,off,1107,adams-walker,ebony,,2021,Baton Rouge PD,2b3884808972f05549fe89ed93cdfff7
...,...,...,...,...,...,...,...,...
605,sgt,170,woodard,rodrick,e,2021,Baton Rouge PD,6d82e73c3d401a0424eef198d7ace669
606,cpl,275,woodring,steven,z,2021,Baton Rouge PD,86ed1bb573d54cc9791cd7adcccb68d5
607,cpl,430,wright,christopher,j,2021,Baton Rouge PD,464b671eff4b54c1765d114ea064d8ac
608,sgt,176,young jr,donald,p,2021,Baton Rouge PD,6503fb0a070a37136a1e291cf7430915


In [3]:
df17

,year,employee_id,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,...,hire_month,hire_day,resign_year,resign_month,resign_day,agency,uid,pay_effective_year,rank_year,dept_year
2629,2008,356565,young,charline,b,5050001,operation services bureau,5130,fingerprint technician,0,...,2,21,,,,Baton Rouge CSD,003c3df3701ee93205c30d9430602764,2008.0,2008.0,2008.0
2630,2009,356565,young,charline,b,5050001,operation services bureau,5130,fingerprint technician,0,...,2,21,,,,Baton Rouge CSD,003c3df3701ee93205c30d9430602764,2009.0,NaN,NaN
2631,2010,356565,young,charline,b,5050001,operation services bureau,5130,fingerprint technician,0,...,2,21,,,,Baton Rouge CSD,003c3df3701ee93205c30d9430602764,2010.0,NaN,NaN
2632,2011,356565,young,charline,b,5050001,operation services bureau,5130,fingerprint technician,0,...,2,21,,,,Baton Rouge CSD,003c3df3701ee93205c30d9430602764,2011.0,NaN,NaN
2633,2012,356565,young,charline,b,5050001,operation services bureau,5130,fingerprint technician,0,...,2,21,,,,Baton Rouge CSD,003c3df3701ee93205c30d9430602764,2012.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,2013,491055,hall,travis,t,5020001,uniform patrol bureau,5005,officer,0,...,8,22,,,,Baton Rouge CSD,ffd61b19fc8c9d5456e34c57201f7e5e,2013.0,NaN,NaN
6585,2014,491055,hall,travis,t,5020001,uniform patrol bureau,5005,officer,0,...,8,22,,,,Baton Rouge CSD,ffd61b19fc8c9d5456e34c57201f7e5e,2014.0,NaN,NaN
7222,2015,491055,hall,travis,t,5020001,uniform patrol bureau,5005,officer,0,...,8,22,,,,Baton Rouge CSD,ffd61b19fc8c9d5456e34c57201f7e5e,2015.0,NaN,NaN
8958,2016,491055,hall,travis,t,5020001,uniform patrol bureau,5005,officer,0,...,8,22,,,,Baton Rouge CSD,ffd61b19fc8c9d5456e34c57201f7e5e,2016.0,NaN,NaN


In [5]:
df19

,year,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,salary,...,hire_month,hire_day,resign_year,resign_month,resign_day,agency,uid,pay_effective_year,rank_year,dept_year
366,2018,davis,ruben,l,5060,special operations bureau,6525,school crossing guard,active,5183.75,...,10,12,,,,Baton Rouge CSD,002a838dd155fd212beff16e1d51d5f2,2018.0,2018.0,2018.0
1298,2019,davis,ruben,l,5060,special operations bureau,6525,school crossing guard,active,4683.50,...,10,12,,,,Baton Rouge CSD,002a838dd155fd212beff16e1d51d5f2,2019.0,NaN,NaN
624,2018,day,louis,r,5070,criminal investigations bureau,5010,sergeant,active,75280.06,...,4,1,,,,Baton Rouge CSD,007dd316e89b0bca62d1e429de66447f,2018.0,2018.0,2018.0
1537,2019,day,louis,r,5070,criminal investigations bureau,5010,sergeant,active,79915.74,...,4,1,,,,Baton Rouge CSD,007dd316e89b0bca62d1e429de66447f,2019.0,NaN,NaN
239,2018,canezaro,scotty,w,5060,special operations bureau,5005,officer,active,62102.91,...,7,9,,,,Baton Rouge CSD,009b9296f30f34e565831502f3a835a0,2018.0,2018.0,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,2019,magee,brett,s,5070,criminal investigations bureau,5010,sergeant,active,66433.79,...,7,31,,,,Baton Rouge CSD,ff74d79f213d679a26f3eb8383b20c08,2019.0,NaN,NaN
800,2018,rogers,christopher,k,5060,special operations bureau,5010,sergeant,active,83372.66,...,7,31,,,,Baton Rouge CSD,ff89ecdef1929840c2cbf3dfe26a9b19,2018.0,2018.0,2018.0
1699,2019,rogers,christopher,k,5060,special operations bureau,5010,sergeant,active,83493.53,...,7,31,,,,Baton Rouge CSD,ff89ecdef1929840c2cbf3dfe26a9b19,2019.0,NaN,NaN
770,2018,taylor,darcy,d,5070,criminal investigations bureau,5005,officer,inactive,57546.46,...,8,31,2018,11,27,Baton Rouge CSD,ffc225cf35e2f4817f1263414f7c72f5,2018.0,2018.0,2018.0


In [22]:
df19[df19.salary.isna()]

,year,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,salary,...,hire_month,hire_day,resign_year,resign_month,resign_day,agency,uid,pay_effective_year,rank_year,dept_year


In [14]:
df2 = df19[['uid', 'employee_id', 'first_name', 'last_name', 'middle_initial']]\
    .drop_duplicates().sort_values('uid')
df2[df2.uid.duplicated(keep=False)].iloc[:30]


,uid,employee_id,first_name,last_name,middle_initial
